# **REGRESSION USING STACKING**

**The data used in the regression: https://www.kaggle.com/datasets/kuchhbhi/latest-laptop-price-list**

**NOTE: Some of the works are helped by documentations**

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('laptop-pricing.csv')
df.head()

,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,...,display_size,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
0,Lenovo,A6-9225,AMD,A6-9225 Processor,10th,4 GB GB,DDR4,0 GB,1024 GB,Windows,...,Missing,0,No,No,24990,32790,23,3.7,63,12
1,Lenovo,Ideapad,AMD,APU Dual,10th,4 GB GB,DDR4,0 GB,512 GB,Windows,...,Missing,0,No,No,19590,21325,8,3.6,1894,256
2,Avita,PURA,AMD,APU Dual,10th,4 GB GB,DDR4,128 GB,0 GB,Windows,...,Missing,0,No,No,19990,27990,28,3.7,1153,159
3,Avita,PURA,AMD,APU Dual,10th,4 GB GB,DDR4,128 GB,0 GB,Windows,...,Missing,0,No,No,21490,27990,23,3.7,1153,159
4,Avita,PURA,AMD,APU Dual,10th,4 GB GB,DDR4,256 GB,0 GB,Windows,...,Missing,0,No,No,24990,33490,25,3.7,1657,234


In [3]:
df.shape

(896, 23)

In [4]:
df.columns

Index(['brand', 'model', 'processor_brand', 'processor_name',
       'processor_gnrtn', 'ram_gb', 'ram_type', 'ssd', 'hdd', 'os', 'os_bit',
       'graphic_card_gb', 'weight', 'display_size', 'warranty', 'Touchscreen',
       'msoffice', 'latest_price', 'old_price', 'discount', 'star_rating',
       'ratings', 'reviews'],
      dtype='object')

**Drop unimportant columns**

In [5]:
df = df.drop('brand', axis = 1)
df = df.drop('model', axis = 1)

**Column renaming**

In [6]:
df.rename(columns = {'Touchscreen': 'touchscreen'}, inplace = True)
df.rename(columns = {'processor_gnrtn': 'processor_generation'}, inplace = True)

**Check the data's condition**

In [7]:
df.shape

(896, 21)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   processor_brand       896 non-null    object 
 1   processor_name        896 non-null    object 
 2   processor_generation  896 non-null    object 
 3   ram_gb                896 non-null    object 
 4   ram_type              896 non-null    object 
 5   ssd                   896 non-null    object 
 6   hdd                   896 non-null    object 
 7   os                    896 non-null    object 
 8   os_bit                896 non-null    object 
 9   graphic_card_gb       896 non-null    int64  
 10  weight                896 non-null    object 
 11  display_size          896 non-null    object 
 12  warranty              896 non-null    int64  
 13  touchscreen           896 non-null    object 
 14  msoffice              896 non-null    object 
 15  latest_price          8

The reason why both brand and model columns are dropped because it's not ordinal.

**Check for missing values or unique values**

In [9]:
for (columnName, columnData) in df.items():
    na_count = columnData.isna().sum()
    unique_count = columnData.unique()
    print(f'{columnName}\'s data type: {columnData.dtypes}')
    print(f'N/A values in {columnName}: {na_count}')
    print(f'{columnName} values: {unique_count}\n')

processor_brand's data type: object
N/A values in processor_brand: 0
processor_brand values: ['AMD' 'Intel' 'M1' 'MediaTek' 'Qualcomm']

processor_name's data type: object
N/A values in processor_name: 0
processor_name values: ['A6-9225 Processor' 'APU Dual' 'Athlon Dual' 'Core i3' 'Core i5'
 'Celeron Dual' 'Ryzen' 'Ryzen 5' 'Core' 'Core i7' 'Core i9' 'Core m3'
 'Dual Core' 'Ever Screenpad' 'GeForce GTX' 'GeForce RTX' 'GEFORCE RTX'
 'Hexa Core' 'M1' 'Genuine Windows' 'Pentium Quad' 'Pentium Silver'
 'Ryzen 3' 'MediaTek Kompanio' 'Quad' 'Ryzen 7' 'Ryzen 9' 'Snapdragon 7c']

processor_generation's data type: object
N/A values in processor_generation: 0
processor_generation values: ['10th' 'Missing' '11th' '7th' '8th' '9th' '4th' '12th']

ram_gb's data type: object
N/A values in ram_gb: 0
ram_gb values: ['4 GB GB' '8 GB GB' '32 GB GB' '16 GB GB']

ram_type's data type: object
N/A values in ram_type: 0
ram_type values: ['DDR4' 'LPDDR4X' 'LPDDR4' 'DDR5' 'DDR3' 'LPDDR3']

ssd's data type: ob

**Tidy up the data**

Text mining (extract numbers from every row in a specified column) and type changing.

In [10]:
df['processor_generation'] = df['processor_generation'].str.extract('(\d+)')
df['ram_gb'] = df['ram_gb'].str.extract('(\d+)')
df['ssd'] = df['ssd'].str.extract('(\d+)')
df['hdd'] = df['hdd'].str.extract('(\d+)')
df['os_bit'] = df['os_bit'].str.extract('(\d+)')

df['processor_generation'] = pd.to_numeric(df['processor_generation'])
df['ram_gb'] = pd.to_numeric(df['ram_gb'])
df['ssd'] = pd.to_numeric(df['ssd'])
df['hdd'] = pd.to_numeric(df['hdd'])
df['os_bit'] = pd.to_numeric(df['os_bit'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   processor_brand       896 non-null    object 
 1   processor_name        896 non-null    object 
 2   processor_generation  657 non-null    float64
 3   ram_gb                896 non-null    int64  
 4   ram_type              896 non-null    object 
 5   ssd                   896 non-null    int64  
 6   hdd                   896 non-null    int64  
 7   os                    896 non-null    object 
 8   os_bit                896 non-null    int64  
 9   graphic_card_gb       896 non-null    int64  
 10  weight                896 non-null    object 
 11  display_size          896 non-null    object 
 12  warranty              896 non-null    int64  
 13  touchscreen           896 non-null    object 
 14  msoffice              896 non-null    object 
 15  latest_price          8

Missing value handling

In [11]:
df[df['processor_name'] == 'Ryzen']

,processor_brand,processor_name,processor_generation,ram_gb,ram_type,ssd,hdd,os,os_bit,graphic_card_gb,...,display_size,warranty,touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
19,AMD,Ryzen,10.0,4,DDR4,0,512,Windows,32,4,...,Missing,0,No,No,54990,89000,38,4.5,3190,414


In [12]:
df[df['processor_name'] == 'Core']

,processor_brand,processor_name,processor_generation,ram_gb,ram_type,ssd,hdd,os,os_bit,graphic_card_gb,...,display_size,warranty,touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
213,Intel,Core,NaN,4,DDR4,0,512,Windows,32,0,...,Missing,0,No,No,99990,149999,33,4.5,1336,186


Since there are only two uninformative processor name, both data are going to be dropped.

In [13]:
df = df.drop(df[df['processor_name'] == 'Ryzen'].index)
df = df.drop(df[df['processor_name'] == 'Core'].index)

#re-check
df[df['processor_name'] == 'Core']

,processor_brand,processor_name,processor_generation,ram_gb,ram_type,ssd,hdd,os,os_bit,graphic_card_gb,...,display_size,warranty,touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews


In [14]:
df['processor_generation'] = df['processor_generation'].fillna(np.mean(df['processor_generation']))

In [15]:
df['display_size'] = df['display_size'].replace('Missing', np.NaN)
df['display_size'] = pd.to_numeric(df['display_size'])
df['display_size'] = df['display_size'].fillna(np.mean(df['display_size']))

In [16]:
for (columnName, columnData) in df.items():
    na_count = columnData.isna().sum()
    unique_count = columnData.unique()
    print(f'{columnName}\'s data type: {columnData.dtypes}')
    print(f'N/A values in {columnName}: {na_count}')
    print(f'{columnName} values: {unique_count}\n')

processor_brand's data type: object
N/A values in processor_brand: 0
processor_brand values: ['AMD' 'Intel' 'M1' 'MediaTek' 'Qualcomm']

processor_name's data type: object
N/A values in processor_name: 0
processor_name values: ['A6-9225 Processor' 'APU Dual' 'Athlon Dual' 'Core i3' 'Core i5'
 'Celeron Dual' 'Ryzen 5' 'Core i7' 'Core i9' 'Core m3' 'Dual Core'
 'Ever Screenpad' 'GeForce GTX' 'GeForce RTX' 'GEFORCE RTX' 'Hexa Core'
 'M1' 'Genuine Windows' 'Pentium Quad' 'Pentium Silver' 'Ryzen 3'
 'MediaTek Kompanio' 'Quad' 'Ryzen 7' 'Ryzen 9' 'Snapdragon 7c']

processor_generation's data type: float64
N/A values in processor_generation: 0
processor_generation values: [10.         10.33231707 11.          7.          8.          9.
  4.         12.        ]

ram_gb's data type: int64
N/A values in ram_gb: 0
ram_gb values: [ 4  8 32 16]

ram_type's data type: object
N/A values in ram_type: 0
ram_type values: ['DDR4' 'LPDDR4X' 'LPDDR4' 'DDR5' 'DDR3' 'LPDDR3']

ssd's data type: int64
N/A val

Binary and label encoding:

In [17]:
from sklearn import preprocessing
label_encoding = preprocessing.LabelEncoder()

In [18]:
df['processor_brand'] = label_encoding.fit_transform(df['processor_brand'])

In [19]:
# #Replace GEFORCE RTX to GeForce RTX
df['processor_name'].replace('GEFORCE RTX', 'GeForce RTX', inplace = True)
df['processor_name'] = label_encoding.fit_transform(df['processor_name'])

C:\Users\steam\AppData\Local\Temp\ipykernel_10880\850789236.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['processor_name'].replace('GEFORCE RTX', 'GeForce RTX', inplace = True)


In [20]:
df['ram_type'] = label_encoding.fit_transform(df['ram_type'])

In [21]:
df['os'] = label_encoding.fit_transform(df['os'])

In [22]:
df['weight'] = label_encoding.fit_transform(df['weight'])

In [23]:
df['touchscreen'] = label_encoding.fit_transform(df['touchscreen'])

In [24]:
df['msoffice'] = label_encoding.fit_transform(df['msoffice'])

In [25]:
df.head()

,processor_brand,processor_name,processor_generation,ram_gb,ram_type,ssd,hdd,os,os_bit,graphic_card_gb,...,display_size,warranty,touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
0,0,0,10.0,4,1,0,1024,2,64,0,...,15.122021,0,0,0,24990,32790,23,3.7,63,12
1,0,1,10.0,4,1,0,512,2,64,0,...,15.122021,0,0,0,19590,21325,8,3.6,1894,256
2,0,1,10.0,4,1,128,0,2,64,0,...,15.122021,0,0,0,19990,27990,28,3.7,1153,159
3,0,1,10.0,4,1,128,0,2,64,0,...,15.122021,0,0,0,21490,27990,23,3.7,1153,159
4,0,1,10.0,4,1,256,0,2,64,0,...,15.122021,0,0,0,24990,33490,25,3.7,1657,234


**Data modelling**

In [26]:
df_input = df.drop('latest_price', axis = 1)
df_output = df['latest_price']

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_input, df_output, test_size = 0.2, random_state = 0)

In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

estimators = [

    ('rf', RandomForestRegressor(random_state=0)), # Random forest as estimator
    ('svr', make_pipeline(StandardScaler(),LinearSVR(random_state=0))) # SVM as estimator

]

In [29]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

stacking = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stacking.fit(x_train, y_train)
y_predict=stacking.predict(x_test)

In [30]:
from sklearn import metrics
from sklearn.metrics import r2_score
print("R2: ", r2_score(y_predict,y_test))
print("MSE: ", metrics.mean_squared_error(y_predict,y_test))
print("RMSE: ", metrics.mean_squared_error(y_predict,y_test) ** 0.5)
print("MAE: ", metrics.mean_absolute_error(y_predict,y_test))

R2:  0.9461095635328698
MSE:  127650622.26789033
RMSE:  11298.257488121359
MAE:  3990.0817128808944
